# Produce Final Reddit Dataset

In [1]:
import gzip
import json
from copy import deepcopy
from tqdm import tqdm

DATA_DIR = '/mnt/ceph/storage/data-in-progress/data-research/web-search/false-memories/reddit-tomt/tomt-dataset-03-12-2022/'

def load_json_data(file_name):
    with gzip.open(DATA_DIR + file_name, 'r') as f:
        for l in tqdm(f):
            try:
                yield json.loads(l)
            except:
                continue



In [2]:
parent_id_to_comments = {}

for i in load_json_data('reddit-tomt-comments.jsonl.gz'):
    if i['parent_id'] not in parent_id_to_comments:
        parent_id_to_comments[i['parent_id']] = []
    
    parent_id_to_comments[i['parent_id']] += [i]

16627156it [05:07, 54090.54it/s]


In [5]:
def get_comments(doc, depth):
    ret = []

    if depth > 10:
        return []
    
    for parent_id in ['id', 'name']:
        if doc and parent_id in doc and len(ret) < 100:
            for prefix in ['', 't1_', 't2_', 't3_', 't4_']:
                ret += parent_id_to_comments.get(prefix + doc[parent_id], [])

    ret_without_duplicates = []
    existing_ids = set()
    
    for i in ret:
        if i['id'] in existing_ids:
            continue
        existing_ids.add(i['id'])
        ret_without_duplicates += [i]
        
    return ret_without_duplicates

def enrich_tomt_with_comments(doc, depth):
    doc = deepcopy(doc)
    
    comments = get_comments(doc, depth)
    comments = [enrich_tomt_with_comments(i, depth+1) for i in comments]
    doc['comments'] = comments
    
    return doc

In [6]:
with gzip.open(DATA_DIR + 'reddit-tomt-submissions-with-comments.jsonl.gz', 'w') as f:
    for tomt in load_json_data('reddit-tomt-submissions.jsonl.gz'):
        tomt = enrich_tomt_with_comments(tomt, 0)
        f.write((json.dumps(tomt) + '\n').encode('utf8'))


2558850it [14:10, 3007.38it/s]


In [7]:
print('Done')

Done
